# Process control with the lucullus REST API

## Introduction

The basic control scheme is often described as a cycle consisting of three repeating steps.

1. Measure
2. Decide
3. Act

So if we want to use this is as a guideline for our own implementation of controlling processes with the REST API, we can also define three simple steps:

1. Collect port data
2. Calculate some stuff
3. Update ports

For this we have created a class "Controller", which does just that and now this notebook will guide you step by step through the process of creating your own simple controller.

In [4]:
from lucullus_rest.core import Controller

auth = ("user", "password")
process = "Process_555"
port_names = ["PV_Temp", "PV_rpm"]

## Use Case I - Only Monitoring Data

So lets say we only want to collect the data with each iteration, a little bit of a constructed case, but still. We create a new instance of the Controller class by initializing it with the necessary information of process name, list of ports which are needed, and authentication. Afterwards we collect the data in our update, and can now print the data that we gathered. As you can see, during each update step we collect the whole timeseries and not only the newest data. This is meant in the case there would be a delay of historical data, for instance in the case of offline or atline analytics.

In [5]:
c = Controller(process, port_names, auth, interp_interval=60)
c.update()

print(c.collected_data)

   Time [h]  PV_rpm    PV_Temp
0  0.001465       0  25.161243


## Use Case II - Monitoring and Calculation

In [6]:
def calc_temp_integral(collected_data, calculated_data):
    collected_data["PV_Temp"].rolling(2).sum()*collected_data["Time [h]"].diff()
    return calculated_data

c = Controller(process, port_names, auth, 
    interp_interval=60,
    calc_fun=calc_temp_integral)

c.update()

print(c.calculated_data)

None


## Use Case III - Monitoring, Calculation, Control

We will now create a simple toy controller, by adding an output function. Our output function gets both the collected and the calculated data as inputs and passes a dictionary with the ports it should update as output. As this is only supposed to show the capabilites of the controller class we will make a simple controller, that switches the temperature setpoint every 5 minutes 1 degree up or down.

In [7]:
import numpy as np
from datetime import datetime

def set_temperature(collected_data, calculated_data):
    # Get current process time
    process_time = collected_data["Time [h]"].values[-1]
    # Define the time in hours when the temperature should be changed
    cycle_time = 1/60*5
    cycle = np.floor(process_time/cycle_time)
    # Define the new temperature as a function of cycle time
    new_temp = 27.5 + 0.5*(-1)**cycle
    updated_ports = {"SP_Temp": new_temp}
    return updated_ports

c = Controller(process, port_names, auth, 
    interp_interval=60,
    update_interval=30,
    calc_fun=calc_temp_integral,
    output_fun=set_temperature)

c.update()

print(c.calculated_data)

2022-05-13 06:41:23.808802 New temperature setpoint: 28.0
None


You can now observe in the Lucullus execution monitor that the temperature should change in your process. Before we have only updated a single time, but if we want to update all 30 seconds, as defined by the parameter update_interval, we start an indefinite update cycle.

In [8]:
c.start_update_cycle()

2022-05-13 06:41:36.229449 New temperature setpoint: 28.0
2022-05-13 06:42:06.396407 New temperature setpoint: 28.0
2022-05-13 06:42:36.355024 New temperature setpoint: 28.0
2022-05-13 06:43:06.305388 New temperature setpoint: 28.0
2022-05-13 06:43:36.368495 New temperature setpoint: 28.0
2022-05-13 06:44:06.721551 New temperature setpoint: 28.0
2022-05-13 06:44:36.441813 New temperature setpoint: 28.0
2022-05-13 06:45:06.248357 New temperature setpoint: 28.0
2022-05-13 06:45:36.569970 New temperature setpoint: 28.0
2022-05-13 06:46:06.431386 New temperature setpoint: 27.0
2022-05-13 06:46:36.562197 New temperature setpoint: 27.0
2022-05-13 06:47:06.388438 New temperature setpoint: 27.0
2022-05-13 06:47:36.554380 New temperature setpoint: 27.0
2022-05-13 06:48:06.551508 New temperature setpoint: 27.0
2022-05-13 06:48:36.579257 New temperature setpoint: 27.0
2022-05-13 06:49:06.578799 New temperature setpoint: 27.0
2022-05-13 06:49:36.525927 New temperature setpoint: 27.0
